In [9]:
import torch as t
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__();
        # 第一位表示输入通道数，第二位表示输出的通道数，第三位表示卷积核5*5
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))

10


In [4]:
for name, parameters in net.named_parameters():
    print(name, ':', parameters.size())

conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


In [10]:
input = Variable(t.randn(1, 1, 32, 32))
out = net(input)
out.size()

torch.Size([1, 10])

In [11]:
net.zero_grad()
out.backward(Variable(t.ones(1, 10)))

In [26]:
output = net(input)
print(output)
target = Variable(t.arange(0., 10.))
print(target)
print(output.data[0])
criterion = nn.MSELoss()
loss = criterion(output.data[0], target)
loss

tensor([[-0.0452,  0.1012, -0.1712,  0.0434,  0.1824,  0.0796, -0.1240, -0.0812,
         -0.1504,  0.0622]], grad_fn=<ThAddmmBackward>)
tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
tensor([-0.0452,  0.1012, -0.1712,  0.0434,  0.1824,  0.0796, -0.1240, -0.0812,
        -0.1504,  0.0622])


tensor(28.7010)

In [28]:
net.zero_grad()
net.conv1.bias.grad
loss.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [30]:
weight = weight - learning_rate * gradient
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

NameError: name 'weight' is not defined